# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [4]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lavrentiya,RU,65.5833,-171.0000,17.71,79,100,4.25,overcast clouds
1,1,Kandrian,PG,-6.2167,149.5500,75.61,89,89,4.03,overcast clouds
2,2,Port Elizabeth,ZA,-33.9180,25.5701,64.69,82,0,20.71,clear sky
3,3,Saskylakh,RU,71.9167,114.0833,-11.40,90,100,10.18,overcast clouds
4,4,Ahipara,NZ,-35.1667,173.1667,62.31,77,26,5.57,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [5]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature (F) you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature (F) you would like for your trip? "))

What is the minimum temperature (F) you would like for your trip? 50
What is the maximum temperature (F) you would like for your trip? 80


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [6]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kandrian,PG,-6.2167,149.5500,75.61,89,89,4.03,overcast clouds
2,2,Port Elizabeth,ZA,-33.9180,25.5701,64.69,82,0,20.71,clear sky
4,4,Ahipara,NZ,-35.1667,173.1667,62.31,77,26,5.57,scattered clouds
5,5,Bluff,NZ,-46.6000,168.3333,58.80,77,51,11.72,broken clouds
6,6,Rikitea,PF,-23.1203,-134.9692,74.52,82,100,15.14,overcast clouds
7,7,Punta Arenas,CL,-53.1500,-70.9167,66.22,51,75,11.50,broken clouds
11,11,Hilo,US,19.7297,-155.0900,77.25,75,75,5.75,broken clouds
14,14,Butaritari,KI,3.0707,172.7902,79.75,81,90,10.02,overcast clouds
16,16,Faya,SA,18.3851,42.4509,60.91,69,20,2.24,few clouds
17,17,Elat,IL,29.5581,34.9482,76.84,53,88,4.61,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [7]:
# Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:

# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
nan_value = float("NaN")
clean_travel_cities = preferred_cities_df.replace("",nan_value)
clean_travel_cities.isnull().sum()
clean_travel_cities.dropna(inplace=True)
clean_travel_cities.isnull().sum()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kandrian,PG,-6.2167,149.5500,75.61,89,89,4.03,overcast clouds
2,2,Port Elizabeth,ZA,-33.9180,25.5701,64.69,82,0,20.71,clear sky
4,4,Ahipara,NZ,-35.1667,173.1667,62.31,77,26,5.57,scattered clouds
5,5,Bluff,NZ,-46.6000,168.3333,58.80,77,51,11.72,broken clouds
6,6,Rikitea,PF,-23.1203,-134.9692,74.52,82,100,15.14,overcast clouds
7,7,Punta Arenas,CL,-53.1500,-70.9167,66.22,51,75,11.50,broken clouds
11,11,Hilo,US,19.7297,-155.0900,77.25,75,75,5.75,broken clouds
14,14,Butaritari,KI,3.0707,172.7902,79.75,81,90,10.02,overcast clouds
16,16,Faya,SA,18.3851,42.4509,60.91,69,20,2.24,few clouds
17,17,Elat,IL,29.5581,34.9482,76.84,53,88,4.61,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [14]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Kandrian,PG,75.61,overcast clouds,-6.2167,149.5500
2,Port Elizabeth,ZA,64.69,clear sky,-33.9180,25.5701
4,Ahipara,NZ,62.31,scattered clouds,-35.1667,173.1667
5,Bluff,NZ,58.80,broken clouds,-46.6000,168.3333
6,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692
7,Punta Arenas,CL,66.22,broken clouds,-53.1500,-70.9167
11,Hilo,US,77.25,broken clouds,19.7297,-155.0900
14,Butaritari,KI,79.75,overcast clouds,3.0707,172.7902
16,Faya,SA,60.91,few clouds,18.3851,42.4509
17,Elat,IL,76.84,overcast clouds,29.5581,34.9482


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [15]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kandrian,PG,75.61,overcast clouds,-6.2167,149.5500,
2,Port Elizabeth,ZA,64.69,clear sky,-33.9180,25.5701,
4,Ahipara,NZ,62.31,scattered clouds,-35.1667,173.1667,
5,Bluff,NZ,58.80,broken clouds,-46.6000,168.3333,
6,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692,
7,Punta Arenas,CL,66.22,broken clouds,-53.1500,-70.9167,
11,Hilo,US,77.25,broken clouds,19.7297,-155.0900,
14,Butaritari,KI,79.75,overcast clouds,3.0707,172.7902,
16,Faya,SA,60.91,few clouds,18.3851,42.4509,
17,Elat,IL,76.84,overcast clouds,29.5581,34.9482,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [16]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [19]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels =requests.get(base_url,params=params).json()
    
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kandrian,PG,75.61,overcast clouds,-6.2167,149.5500,Kandrian Station
2,Port Elizabeth,ZA,64.69,clear sky,-33.9180,25.5701,39 On Nile Guest House
4,Ahipara,NZ,62.31,scattered clouds,-35.1667,173.1667,Ahipara Holiday Park
5,Bluff,NZ,58.80,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
6,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Punta Arenas,CL,66.22,broken clouds,-53.1500,-70.9167,Hotel Hain
11,Hilo,US,77.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Butaritari,KI,79.75,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
16,Faya,SA,60.91,few clouds,18.3851,42.4509,شيلة نسايم
17,Elat,IL,76.84,overcast clouds,29.5581,34.9482,Club Hotel Eilat


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [27]:

# Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
clean_hotel_df = hotel_df.replace("",nan_value)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df.isnull().sum()


# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kandrian,PG,75.61,overcast clouds,-6.2167,149.5500,Kandrian Station
2,Port Elizabeth,ZA,64.69,clear sky,-33.9180,25.5701,39 On Nile Guest House
4,Ahipara,NZ,62.31,scattered clouds,-35.1667,173.1667,Ahipara Holiday Park
5,Bluff,NZ,58.80,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
6,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Punta Arenas,CL,66.22,broken clouds,-53.1500,-70.9167,Hotel Hain
11,Hilo,US,77.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Butaritari,KI,79.75,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
16,Faya,SA,60.91,few clouds,18.3851,42.4509,شيلة نسايم
17,Elat,IL,76.84,overcast clouds,29.5581,34.9482,Club Hotel Eilat


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [28]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kandrian,PG,75.61,overcast clouds,-6.2167,149.5500,Kandrian Station
2,Port Elizabeth,ZA,64.69,clear sky,-33.9180,25.5701,39 On Nile Guest House
4,Ahipara,NZ,62.31,scattered clouds,-35.1667,173.1667,Ahipara Holiday Park
5,Bluff,NZ,58.80,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
6,Rikitea,PF,74.52,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Punta Arenas,CL,66.22,broken clouds,-53.1500,-70.9167,Hotel Hain
11,Hilo,US,77.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Butaritari,KI,79.75,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
16,Faya,SA,60.91,few clouds,18.3851,42.4509,شيلة نسايم
17,Elat,IL,76.84,overcast clouds,29.5581,34.9482,Club Hotel Eilat


### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [32]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [33]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Kandrian Station</dd>\n<dt>City</dt><dd>Kandrian</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 75.61 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>39 On Nile Guest House</dd>\n<dt>City</dt><dd>Port Elizabeth</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 64.69 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Ahipara Holiday Park</dd>\n<dt>City</dt><dd>Ahipara</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 62.31 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bluff Homestead - Guesthouse & Campervan Park</dd>\n<dt>City</dt><dd>Bluff</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>broken clouds and 58.8 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 74.52 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [34]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,-6.2167,149.5500
2,-33.9180,25.5701
4,-35.1667,173.1667
5,-46.6000,168.3333
6,-23.1203,-134.9692
7,-53.1500,-70.9167
11,19.7297,-155.0900
14,3.0707,172.7902
16,18.3851,42.4509
17,29.5581,34.9482


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [35]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))